# Imports

In [17]:
import pandas as pd

In [42]:
def dictToCSV(d, name1, name2, filename):
	node = []
	comm = []
	for key in d:
		node.append(key)
		comm.append(d[key])
	
	df = pd.DataFrame({name1: node, name2: comm})
	df.to_csv(filename, index=False)
	return df

def csvToDict(filename, col1, col2):
	df = pd.read_csv(filename)
	d = {}
	df = df[[col1, col2]]
	for ind in df.index:
		d[df[col1][ind]] = df[col2][ind]
	# print(d)
	return d

# Author to Index and edge list generation

In [2]:
df = pd.read_csv("IEEE VIS papers 1990-2021 - Main dataset.csv")

In [7]:
author_df = df[["AuthorNames"]]
author_df = author_df.dropna()

In [12]:
edgeList = []
authors = {}

In [13]:
count = 0
for ind in author_df.index:
    auth = author_df["AuthorNames"][ind]
    current = auth.split(";")
    for i in range(len(current)):
        for j in range(i+1, len(current)):
            if current[i] not in authors:
                authors[current[i]] = count
                count += 1
            if current[j] not in authors:
                authors[current[j]] = count
                count += 1
            edgeList.append("{} {}\n".format(authors[current[i]], authors[current[j]]))

In [14]:
f = open("edgeList", "w")
f.writelines(edgeList)

In [15]:
f.close()

In [18]:
dictToCSV(authors, "author", "id", "authorToInt.csv")

,author,id
0,Michael Bostock,0
1,Vadim Ogievetsky,1
2,Jeffrey Heer,2
3,B. Johnson,3
4,B. Shneiderman,4
...,...,...
7010,G. Almasi,7010
7011,S. Kuschfeldt,7011
7012,M. Holzner,7012
7013,B.M. Dennis,7013


# Add author id to community file

In [24]:
comm_df = pd.read_csv("layer1/community.csv")

In [25]:
ids = []

In [26]:
for ind in comm_df.index:
    ids.append(authors[comm_df["author"][ind]])

In [27]:
comm_df["id"] = ids

In [29]:
comm_df.describe()

,community,id
count,7015.000000,7015.000000
mean,79.063721,3507.000000
std,112.314052,2025.200402
min,0.000000,0.000000
25%,13.000000,1753.500000
50%,35.000000,3507.000000
75%,67.000000,5260.500000
max,464.000000,7014.000000


In [33]:
comm_df.to_csv("community.csv", index=False)

# Generate intra-community edge files 

## Add inter-community to "other.edge"

In [ ]:
comm_map = csvToDict("community.csv", "id", "community")

In [47]:
intra = {}
inter = []

In [48]:
for edge in edgeList:
    edge = edge[:-1]
    edge = list(map(int, edge.split()))
    if comm_map[edge[0]] == comm_map[edge[1]]:
        if comm_map[edge[0]] not in intra:
            intra[comm_map[edge[0]]] = []
        intra[comm_map[edge[0]]].append("{} {}\n".format(edge[0], edge[1]))
    else:
        inter.append("{} {}\n".format(edge[0], edge[1]))

write other.edge

In [49]:
f = open("community/other.edge", "w")
f.writelines(inter)
f.close()

write \<comm_id\>.edge

In [50]:
for id in intra:
    f = open("community/{}.edge".format(str(id)), "w")
    f.writelines(intra[id])
    f.close()